# This notebook is about making the Test Data workable and processable

In [ ]:
# imports
# I was trying a lot of different things during this notebook therefore there might be the one or two libaries which
# arn´t needed anymore


import requests                 # How Python gets the webpages
from bs4 import BeautifulSoup   # Creates structured, searchable object
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from pandas import ExcelWriter
from pandas import ExcelFile
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter 

import datetime as dt
from datetime import date
from scipy import stats
#from pandas.core import datetools
from plotly import tools
import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.tools as tls
import plotly.graph_objs as go
import warnings
import seaborn as sns


from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn import metrics 
from sklearn import preprocessing


import json
from time import sleep
from datetime import datetime

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import matplotlib

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix # confusion matrix
from sklearn import metrics

import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import  metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import gc

In [2]:
from pylab import rcParams

rcParams['figure.figsize'] = 30, 30
rcParams['font.size'] = 20
rcParams['axes.facecolor'] = 'white'

%matplotlib inline

warnings.filterwarnings("ignore")

# plt.style.available
plt.style.use("seaborn-whitegrid")

In [3]:
#1  conda install -c anaconda lightgbm
#   conda install -c conda-forge r-catboost

# pip install lightgbm
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

import lightgbm as lgb
import xgboost as xgb
#from catboost import CatBoostRegressor

from IPython.display import display # Allows the use of display() for DataFrames

import warnings
warnings.filterwarnings('ignore')

In [234]:
#data_train= pd.read_csv('/Users/marcgehring/Desktop/Massey IT/158333/notebooks/train_v2.csv', nrows = 100000)
#data_test= pd.read_csv('/Users/marcgehring/Desktop/Massey IT/158333/notebooks/ga-customer-revenue-prediction/test_v2.csv')
#sub = pd.read_csv('/Users/marcgehring/Desktop/Massey IT/158333/notebooks/ga-customer-revenue-prediction/sample_submission_v2.csv')

In [3]:
# importing the Jason Data
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize



# Working with the first Dataset

In [255]:
# emplanaition: https://www.kaggle.com/kabure/exploring-the-consumer-patterns-ml-pipeline

csv_path='/Users/marcgehring/Desktop/Massey IT/158333/notebooks/ga-customer-revenue-prediction/train_v2.csv'
nrows=200000
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
df = pd.read_csv(csv_path, converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")

#print(os.listdir("../input"))
data_train = df  # this is our working file. So we can perform on data_train

Loaded train_v2.csv. Shape: (200000, 60)


In [4]:
# emplanaition: https://www.kaggle.com/kabure/exploring-the-consumer-patterns-ml-pipeline

csv_path='/Users/marcgehring/Desktop/Massey IT/158333/notebooks/ga-customer-revenue-prediction/test_v2.csv'
#nrows=300000
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
df = pd.read_csv(csv_path, converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows= None,

                )
    
for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")

#print(os.listdir("../input"))
data_test2 = df  # this is our working file. So we can perform on data_train

Loaded test_v2.csv. Shape: (401589, 59)


In [5]:
# zusätzlich muss das noch bei test gedroppt werden, da das optimierte bei tobi schon in train gedroppt wurde

data_test2.drop(['totals.bounces', 'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.isVideoAd','hits','customDimensions'
       ],     # 'trafficSource.campaignCode' hat er nicht gefunden
       axis= 1, inplace = True)
data_test2.shape

(401589, 50)

# now encoding the big Dataset

In [13]:
original_c =data_test2.columns
original_c

Index(['channelGrouping', 'date', 'fullVisitorId', 'socialEngagementType',
       'visitId', 'visitNumber', 'visitStartTime', 'device.browser',
       'device.browserVersion', 'device.browserSize', 'device.operatingSystem',
       'device.operatingSystemVersion', 'device.isMobile',
       'device.mobileDeviceBranding', 'device.mobileDeviceModel',
       'device.mobileInputSelector', 'device.mobileDeviceInfo',
       'device.mobileDeviceMarketingName', 'device.flashVersion',
       'device.language', 'device.screenColors', 'device.screenResolution',
       'device.deviceCategory', 'geoNetwork.continent',
       'geoNetwork.subContinent', 'geoNetwork.country', 'geoNetwork.region',
       'geoNetwork.metro', 'geoNetwork.city', 'geoNetwork.cityId',
       'geoNetwork.networkDomain', 'geoNetwork.latitude',
       'geoNetwork.longitude', 'geoNetwork.networkLocation', 'totals.visits',
       'totals.hits', 'totals.pageviews', 'totals.timeOnSite',
       'totals.sessionQualityDim', 'totals.new

In [14]:
train_RM = data_test2
del data_test2

In [15]:
for f in train_RM.columns:
   # if f != 'total_transactionRevenue':
        if train_RM[f].dtype=='float': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_RM[f].values))
            train_RM[f] = lbl.transform(list(train_RM[f].values))

In [16]:
for f in train_RM.columns:
   # if f != 'total_transactionRevenue':
        if train_RM[f].dtype=='str': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_RM[f].values))
            train_RM[f] = lbl.transform(list(train_RM[f].values))

In [22]:
train_RM.drop('trafficSource.adwordsClickInfo.criteriaParameters',inplace=True, axis=1)
train_RM.head(1)

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,...,totals.newVisits,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,trafficSource.referralPath,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.isTrueDirect
0,4,20180511,221056,0,1526099341,2,1526099341,25,0,0,...,1,7,1950,2461,0,3,62,5,2,True


In [23]:
# idea f rom thissource: https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor

 # estimators: BayesianRidge(), DecisionTreeRegressor(max_features='sqrt', random_state=0), 
 # ExtraTreesRegressor(n_estimators=10, random_state=0),
 # KNeighborsRegressor(n_neighbors=15)

imp = IterativeImputer(estimator = BayesianRidge(), 
                       missing_values=np.nan, 
                       sample_posterior=False, 
                       max_iter=10, 
                       tol=0.001, 
                       n_nearest_features=10, # amount of features,
                       imputation_order = 'ascending',
                       min_value = None,
                       max_value = None, 
                       verbose = 2, # controls the debug messages 
                       random_state = None,
                       add_indicator = True,
                       initial_strategy='mean')
imp.fit(train_RM)
train_RM = pd.DataFrame(data=imp.transform(train_RM),
                             dtype='int')

[IterativeImputer] Completing matrix with shape (401589, 49)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.95
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (401589, 49)
[IterativeImputer] Ending imputation round 1/1, elapsed time 0.03


In [24]:
for f in train_RM.columns:
    #if f != 'total_transactionRevenue':
        if train_RM[f].dtype=='object': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_RM[f].values))
            train_RM[f] = lbl.transform(list(train_RM[f].values))

In [25]:
train_RM.head(1)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,4,20180511,221056,0,1526099341,2,1526099341,25,0,0,...,7,1950,2461,0,3,62,5,2,1,0


In [27]:
train_RM.
len(original_c)

50

In [ ]:
train_RM.drop(51, axis=1, inplace = True)
train_RM.drop(52, axis=1, inplace = True)
train_RM.drop(53, axis=1, inplace = True)

In [28]:
namesList = original_c

In [29]:
train_RM.columns = namesList

In [31]:
train_RM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401589 entries, 0 to 401588
Data columns (total 50 columns):
channelGrouping                                      401589 non-null int64
date                                                 401589 non-null int64
fullVisitorId                                        401589 non-null int64
socialEngagementType                                 401589 non-null int64
visitId                                              401589 non-null int64
visitNumber                                          401589 non-null int64
visitStartTime                                       401589 non-null int64
device.browser                                       401589 non-null int64
device.browserVersion                                401589 non-null int64
device.browserSize                                   401589 non-null int64
device.operatingSystem                               401589 non-null int64
device.operatingSystemVersion                        401589 non-

In [10]:
data_test2.to_csv('Google_jason_cleaned_data_test_full_final.csv')
